In [29]:
import pandas as pd
import statsmodels.api as sm
import re
import numpy as np
import pandas as pd
from itertools import product

In [30]:
def remove_pattern(col_name):
    return re.sub(r'_[0-9]{3}$', '', col_name)

Zones = pd.read_excel('Zones.xlsx')
Dataset = pd.read_csv('KAG 2016-17\Agriculture\Agriculture_KAG_2016_17.csv')
Dataset.columns = map(remove_pattern, Dataset.columns)

Working_Directory = pd.read_csv('Working_Directory.csv')
Working_Directory[24:]

,District,Rice_Production,Maize_Production,Groundnut_Production,TotalNPK,SowingSeedsDistributed_Maize,LivestockCensus_2012_Cattle,AreaUnderCereals_Maize,AreaUnderCereals_Paddy,NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderOilSeeds_GroundNut,MotorVehicles_Tractors,LivestockCensus_2012_Sheep,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea,Rice_Ratio,Maize_Ratio,Groundnut_Ratio,Admin_Zone
24,BALLARI,206115.5,104045,22938.5,61664.5,2859.005,172038.5,49869.5,46832.5,77607,49762,24001,756270,75198,0.269990,0.136288,0.030047,Kalaburagi Division
25,BIDAR,2036.0,2791,261.0,24658.0,24.550,234519.0,1382.0,1548.0,0,299,5256,88402,19908,0.007378,0.010114,0.000946,Kalaburagi Division
26,KALABURAGI,7888.0,14621,3114.0,53999.0,165.000,473117.0,2634.0,4199.0,20864,3817,16351,100061,48802,0.007095,0.013150,0.002801,Kalaburagi Division
27,YADGIRI,224659.0,2673,31435.0,91389.0,0.000,310727.0,961.0,49358.0,91141,38840,9902,463482,22035,0.339006,0.004033,0.047435,Kalaburagi Division
28,RAICHUR,492342.0,723,26958.0,117235.0,14.350,359124.0,2600.0,115246.0,111512,35960,25612,647549,26571,0.357505,0.000525,0.019575,Kalaburagi Division
29,KOPPAL,243505.0,190252,29982.0,73552.0,5292.000,260408.0,48210.0,50753.0,37736,22274,16317,547061,82552,0.191373,0.149521,0.023563,Kalaburagi Division
30,VIJAYANAGARA,206115.5,104045,22938.5,61664.5,2859.005,172038.5,49869.5,46832.5,77607,49762,24001,756270,75198,0.269990,0.136288,0.030047,Kalaburagi Division


In [31]:
Case_file = pd.read_csv('Params/5_Param_Maize_Multicollinearity.csv')
y = Working_Directory['Maize_Production']
Case_file[:5]

,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182
0,4.45,2567.461578,1129,438.180210,175.723235
1,134.73,25037.139270,14867,3036.080250,3197.193185
2,260.15,7729.614277,1987,222.225258,709.859804
3,5004.00,83821.593090,92194,10902.406570,28105.507250
4,6124.60,85657.632090,173400,25702.835480,20213.730880


In [32]:
if(len(Case_file) < 31):
    row_24 = Case_file.iloc[24]
    halved_row_24 = row_24 / 2
    Case_file.iloc[24] = halved_row_24
    Case_file.loc[len(Case_file)] = halved_row_24

Case_file[24:]

,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182
24,2859.005,23446.847905,49869.5,8404.154610,5124.306675
25,24.550,2371.831678,1382.0,249.381182,201.341576
26,165.000,6221.617910,2634.0,710.101614,641.759644
27,0.000,1253.317144,961.0,368.617469,88.878158
28,14.350,188.537479,2600.0,61.547519,13.949581
29,5292.000,38936.400170,48210.0,10997.550400,12343.237180
30,2859.005,23446.847905,49869.5,8404.154610,5124.306675


In [34]:
# Ensure that the index of y and Case_file are aligned
y = y.loc[Case_file.index]

# Identify rows in Case_file that have any zeros
rows_to_remove = (Case_file == 0).any(axis=1)

# Apply the mask to both DataFrames
Case_file = Case_file[~rows_to_remove].reset_index(drop=True)
y = y[~rows_to_remove].reset_index(drop=True)

# Correct way to get the length of Case_file
case_file_length = len(Case_file)
case_file_length

28

In [35]:
Case_file =sm.add_constant(Case_file)
Case_file.head()

,const,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182
0,1.0,4.45,2567.461578,1129.0,438.180210,175.723235
1,1.0,134.73,25037.139270,14867.0,3036.080250,3197.193185
2,1.0,260.15,7729.614277,1987.0,222.225258,709.859804
3,1.0,5004.00,83821.593090,92194.0,10902.406570,28105.507250
4,1.0,6124.60,85657.632090,173400.0,25702.835480,20213.730880


In [36]:
model=sm.OLS(y,Case_file).fit()
print(model.summary())
model.params.values
coef=model.params.values[1:]
coef

                            OLS Regression Results                            
Dep. Variable:       Maize_Production   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     100.3
Date:                Mon, 16 Sep 2024   Prob (F-statistic):           2.18e-14
Time:                        19:56:25   Log-Likelihood:                -331.18
No. Observations:                  28   AIC:                             674.4
Df Residuals:                      22   BIC:                             682.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

array([-10.26475263,   0.81677644,   2.0182737 ,   6.5872575 ,
         0.47825543])

In [37]:
Case_file.drop('const',axis=1,inplace=True)
Case_file[:5]

,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182
0,4.45,2567.461578,1129.0,438.180210,175.723235
1,134.73,25037.139270,14867.0,3036.080250,3197.193185
2,260.15,7729.614277,1987.0,222.225258,709.859804
3,5004.00,83821.593090,92194.0,10902.406570,28105.507250
4,6124.60,85657.632090,173400.0,25702.835480,20213.730880


In [38]:
Case_file.loc[:,'Beta_0'] = model.params.values[0]
Case_file.loc[:,'Beta_1'] = coef[0]
Case_file.loc[:,'Beta_2'] = coef[1]
Case_file.loc[:,'Beta_3'] = coef[2]
Case_file.loc[:,'Beta_4'] = coef[3]
Case_file.loc[:,'Beta_5'] = coef[4]

intervals = np.arange(0, 1.1, 0.1)
columns=['Sensitivity_SowingSeedsDistributed_Maize_320', 'Sensitivity_LivestockCensus_2012_Cattle_354' , 'Sensitivity_AreaUnderCereals_Maize_193', 'Sensitivity_TotalNPK_315','Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182']
# Generate all possible combinations of elements in arr
combinations = product(intervals, repeat=len(columns))
combinations

In [39]:
alpha_df = pd.DataFrame(combinations, columns=columns)
alpha_df

,Sensitivity_SowingSeedsDistributed_Maize_320,Sensitivity_LivestockCensus_2012_Cattle_354,Sensitivity_AreaUnderCereals_Maize_193,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.1
2,0.0,0.0,0.0,0.0,0.2
3,0.0,0.0,0.0,0.0,0.3
4,0.0,0.0,0.0,0.0,0.4
...,...,...,...,...,...
161046,1.0,1.0,1.0,1.0,0.6
161047,1.0,1.0,1.0,1.0,0.7
161048,1.0,1.0,1.0,1.0,0.8
161049,1.0,1.0,1.0,1.0,0.9


In [40]:
Case_file['key'] = 1
alpha_df['key'] = 1

# Performing the Cartesian product
result_df = pd.merge( alpha_df, Case_file, on='key').drop('key', axis=1)

print(result_df)

         Sensitivity_SowingSeedsDistributed_Maize_320  \
0                                                 0.0   
1                                                 0.0   
2                                                 0.0   
3                                                 0.0   
4                                                 0.0   
...                                               ...   
4509423                                           1.0   
4509424                                           1.0   
4509425                                           1.0   
4509426                                           1.0   
4509427                                           1.0   

         Sensitivity_LivestockCensus_2012_Cattle_354  \
0                                                0.0   
1                                                0.0   
2                                                0.0   
3                                                0.0   
4                                  

In [41]:
result_df.head()

,Sensitivity_SowingSeedsDistributed_Maize_320,Sensitivity_LivestockCensus_2012_Cattle_354,Sensitivity_AreaUnderCereals_Maize_193,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182,Beta_0,Beta_1,Beta_2,Beta_3,Beta_4,Beta_5
0,0.0,0.0,0.0,0.0,0.0,4.45,2567.461578,1129.0,438.180210,175.723235,-8388.294407,-10.264753,0.816776,2.018274,6.587258,0.478255
1,0.0,0.0,0.0,0.0,0.0,134.73,25037.139270,14867.0,3036.080250,3197.193185,-8388.294407,-10.264753,0.816776,2.018274,6.587258,0.478255
2,0.0,0.0,0.0,0.0,0.0,260.15,7729.614277,1987.0,222.225258,709.859804,-8388.294407,-10.264753,0.816776,2.018274,6.587258,0.478255
3,0.0,0.0,0.0,0.0,0.0,5004.00,83821.593090,92194.0,10902.406570,28105.507250,-8388.294407,-10.264753,0.816776,2.018274,6.587258,0.478255
4,0.0,0.0,0.0,0.0,0.0,6124.60,85657.632090,173400.0,25702.835480,20213.730880,-8388.294407,-10.264753,0.816776,2.018274,6.587258,0.478255


In [42]:
Working_Directory['Maize_Production'].mean()

134853.12903225806

In [48]:
delta_O = 5000

result_df['New_F1'] = result_df['SowingSeedsDistributed_Maize_320'] + (result_df['Sensitivity_SowingSeedsDistributed_Maize_320']*delta_O)/result_df['Beta_1']
result_df['New_F2'] = result_df['LivestockCensus_2012_Cattle_354'] + (result_df['Sensitivity_LivestockCensus_2012_Cattle_354']*delta_O)/result_df['Beta_2']
result_df['New_F3'] = result_df['AreaUnderCereals_Maize_193'] + (result_df['Sensitivity_AreaUnderCereals_Maize_193']*delta_O)/result_df['Beta_3']
result_df['New_F4'] = result_df['TotalNPK_315'] + (result_df['Sensitivity_TotalNPK_315']*delta_O)/result_df['Beta_4']
result_df['New_F5'] = result_df['NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182'] + (result_df['Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182']*delta_O)/result_df['Beta_5']
result_df['District'] = np.tile(Working_Directory[~rows_to_remove]['District'],11**5)

result_df['New_F1'] = result_df['New_F1'].apply(lambda x: max(0, x))
result_df['New_F2'] = result_df['New_F2'].apply(lambda x: max(0, x))
result_df['New_F3'] = result_df['New_F3'].apply(lambda x: max(0, x))
result_df['New_F4'] = result_df['New_F4'].apply(lambda x: max(0, x))
result_df['New_F5'] = result_df['New_F5'].apply(lambda x: max(0, x))

result_df

,Sensitivity_SowingSeedsDistributed_Maize_320,Sensitivity_LivestockCensus_2012_Cattle_354,Sensitivity_AreaUnderCereals_Maize_193,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182,...,Beta_2,Beta_3,Beta_4,Beta_5,New_F1,New_F2,New_F3,New_F4,New_F5,District
0,0.0,0.0,0.0,0.0,0.0,4.450,2567.461578,1129.0,438.180210,175.723235,...,0.816776,2.018274,6.587258,0.478255,4.450000,2567.461578,1129.000000,438.180210,175.723235,BENGALURU
1,0.0,0.0,0.0,0.0,0.0,134.730,25037.139270,14867.0,3036.080250,3197.193185,...,0.816776,2.018274,6.587258,0.478255,134.730000,25037.139270,14867.000000,3036.080250,3197.193185,BENGALURU(R)
2,0.0,0.0,0.0,0.0,0.0,260.150,7729.614277,1987.0,222.225258,709.859804,...,0.816776,2.018274,6.587258,0.478255,260.150000,7729.614277,1987.000000,222.225258,709.859804,RAMANAGARA
3,0.0,0.0,0.0,0.0,0.0,5004.000,83821.593090,92194.0,10902.406570,28105.507250,...,0.816776,2.018274,6.587258,0.478255,5004.000000,83821.593090,92194.000000,10902.406570,28105.507250,CHITRADURGA
4,0.0,0.0,0.0,0.0,0.0,6124.600,85657.632090,173400.0,25702.835480,20213.730880,...,0.816776,2.018274,6.587258,0.478255,6124.600000,85657.632090,173400.000000,25702.835480,20213.730880,DAVANAGERE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4509423,1.0,1.0,1.0,1.0,1.0,24.550,2371.831678,1382.0,249.381182,201.341576,...,0.816776,2.018274,6.587258,0.478255,0.000000,8493.457838,3859.364688,1008.422407,10656.005959,BIDAR
4509424,1.0,1.0,1.0,1.0,1.0,165.000,6221.617910,2634.0,710.101614,641.759644,...,0.816776,2.018274,6.587258,0.478255,0.000000,12343.244070,5111.364688,1469.142839,11096.424027,KALABURAGI
4509425,1.0,1.0,1.0,1.0,1.0,14.350,188.537479,2600.0,61.547519,13.949581,...,0.816776,2.018274,6.587258,0.478255,0.000000,6310.163639,5077.364688,820.588744,10468.613965,RAICHUR
4509426,1.0,1.0,1.0,1.0,1.0,5292.000,38936.400170,48210.0,10997.550400,12343.237180,...,0.816776,2.018274,6.587258,0.478255,4804.896201,45058.026330,50687.364688,11756.591624,22797.901563,KOPPAL


In [49]:
result_df['New_Production'] = result_df['Beta_1'] * result_df['New_F1'] + result_df['Beta_2'] * result_df['New_F2'] + result_df['Beta_3'] * result_df['New_F3'] + result_df['Beta_0']
result_df

,Sensitivity_SowingSeedsDistributed_Maize_320,Sensitivity_LivestockCensus_2012_Cattle_354,Sensitivity_AreaUnderCereals_Maize_193,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182,SowingSeedsDistributed_Maize_320,LivestockCensus_2012_Cattle_354,AreaUnderCereals_Maize_193,TotalNPK_315,NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182,...,Beta_3,Beta_4,Beta_5,New_F1,New_F2,New_F3,New_F4,New_F5,District,New_Production
0,0.0,0.0,0.0,0.0,0.0,4.450,2567.461578,1129.0,438.180210,175.723235,...,2.018274,6.587258,0.478255,4.450000,2567.461578,1129.000000,438.180210,175.723235,BENGALURU,-4058.299429
1,0.0,0.0,0.0,0.0,0.0,134.730,25037.139270,14867.0,3036.080250,3197.193185,...,2.018274,6.587258,0.478255,134.730000,25037.139270,14867.000000,3036.080250,3197.193185,BENGALURU(R),40684.155987
2,0.0,0.0,0.0,0.0,0.0,260.150,7729.614277,1987.0,222.225258,709.859804,...,2.018274,6.587258,0.478255,260.150000,7729.614277,1987.000000,222.225258,709.859804,RAMANAGARA,-734.993155
3,0.0,0.0,0.0,0.0,0.0,5004.000,83821.593090,92194.0,10902.406570,28105.507250,...,2.018274,6.587258,0.478255,5004.000000,83821.593090,92194.000000,10902.406570,28105.507250,CHITRADURGA,194783.111197
4,0.0,0.0,0.0,0.0,0.0,6124.600,85657.632090,173400.0,25702.835480,20213.730880,...,2.018274,6.587258,0.478255,6124.600000,85657.632090,173400.000000,25702.835480,20213.730880,DAVANAGERE,348675.997024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4509423,1.0,1.0,1.0,1.0,1.0,24.550,2371.831678,1382.0,249.381182,201.341576,...,2.018274,6.587258,0.478255,0.000000,8493.457838,3859.364688,1008.422407,10656.005959,BIDAR,6338.216074
4509424,1.0,1.0,1.0,1.0,1.0,165.000,6221.617910,2634.0,710.101614,641.759644,...,2.018274,6.587258,0.478255,0.000000,12343.244070,5111.364688,1469.142839,11096.424027,KALABURAGI,12009.509428
4509425,1.0,1.0,1.0,1.0,1.0,14.350,188.537479,2600.0,61.547519,13.949581,...,2.018274,6.587258,0.478255,0.000000,6310.163639,5077.364688,820.588744,10468.613965,RAICHUR,7013.210188
4509426,1.0,1.0,1.0,1.0,1.0,5292.000,38936.400170,48210.0,10997.550400,12343.237180,...,2.018274,6.587258,0.478255,4804.896201,45058.026330,50687.364688,11756.591624,22797.901563,KOPPAL,81393.944018


In [50]:
def normalize_list(lst):
    min_val = min(lst)
    max_val = max(lst)
    normalized_lst = [(x - min_val) / (max_val - min_val) for x in lst]
    return normalized_lst

In [51]:
import pandas as pd
import numpy as np

adjacency_excel = pd.read_excel('Karnataka_District_Adjacency_File.xlsx')

district_codes = sorted(set(adjacency_excel['District_Code']))

n = len(district_codes)
adj_matrix = np.zeros((n, n), dtype=int)

code_to_index = {code: idx for idx, code in enumerate(district_codes)}

for i in range(len(adjacency_excel)):
    district_code = adjacency_excel['District_Code'][i]
    neighbors = [int(num) for num in str(adjacency_excel['District_Neighbors'][i]).split(',') if num.strip().isdigit()]
    
    for neighbor in neighbors:
        adj_matrix[code_to_index[district_code]][code_to_index[neighbor]] = 1
        adj_matrix[code_to_index[neighbor]][code_to_index[district_code]] = 1  # Assuming adjacency is bidirectional

adj_matrix_df = pd.DataFrame(adj_matrix, index=district_codes, columns=district_codes)

print(adj_matrix_df)

adj_matrix_np = adj_matrix_df.to_numpy()

    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
1    0   1   1   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2    1   0   1   0   0   1   1   0   1   0  ...   0   0   0   0   0   0   0   
3    1   1   0   0   0   0   0   0   1   0  ...   0   0   0   0   0   0   0   
4    0   0   0   0   1   0   0   0   1   1  ...   0   0   0   1   0   0   0   
5    0   0   0   1   0   0   0   1   0   1  ...   0   1   0   1   0   0   0   
6    0   1   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   
7    0   1   0   0   0   1   0   0   1   0  ...   0   0   0   0   0   0   0   
8    0   0   0   0   1   0   0   0   0   1  ...   0   1   1   0   0   0   0   
9    0   1   1   1   0   0   1   0   0   1  ...   0   0   0   0   0   0   0   
10   0   0   0   1   1   0   0   1   1   0  ...   0   0   0   0   0   0   0   
11   0   0   0   0   0   0   0   0   0   1  ...   0   0   0   0   0   0   0   
12   0   0   0   0   0   0   0   1   0   1  ...   0 

In [52]:
def calculate_avg_capability(capability, neighbors):
    num_neighbors = len(neighbors)
    if num_neighbors == 0:
        return 0
    sum_capabilities = sum(capability[n] for n in neighbors)
    return sum_capabilities / num_neighbors

def calculate_stress_stability(capability, adjacency_matrix):
    n = len(capability)
    stress = np.zeros(n)
    stability = np.zeros(n)
    
    for i in range(n):
        neighbors = [j for j in range(n) if adjacency_matrix[i][j] == 1 and i != j]
        if len(neighbors) > 0:
            avg_capability = calculate_avg_capability(capability, neighbors)
            stress[i] = abs(avg_capability - capability[i])
            stability[i] = 1 - stress[i]
        else:
            stress[i] = 0  # If no neighbors, stress is zero
            stability[i] = 1
        
    return stress, stability

In [41]:
result_df['Normalized_New_Prouction'] = normalize_list(result_df['New_Production'])
result_df

,Sensitivity_AreaUnderCereals_Paddy_190,Sensitivity_TotalNPK_315,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated_173,AreaUnderCereals_Paddy_190,TotalNPK_315,NetAreaIrrigated_Canals_NetAreaIrrigated_173,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District,New_Production,Normalized_New_Prouction
0,0.0,0.0,0.0,782.0,513.822990,0.000000,3.159514,0.987727,2.646305,782.000000,513.822990,0.000000,BENGALURU,2978.256785,0.005455
1,0.0,0.0,0.0,736.0,167.835355,0.000000,3.159514,0.987727,2.646305,736.000000,167.835355,0.000000,BENGALURU(R),2491.177898,0.004529
2,0.0,0.0,0.0,4468.0,257.908893,272.509463,3.159514,0.987727,2.646305,4468.000000,257.908893,272.509463,RAMANAGARA,15092.595963,0.028490
3,0.0,0.0,0.0,1297.0,183.032957,9.283245,3.159514,0.987727,2.646305,1297.000000,183.032957,9.283245,CHITRADURGA,4303.242726,0.007975
4,0.0,0.0,0.0,98259.0,21197.529550,18037.386430,3.159514,0.987727,2.646305,98259.000000,21197.529550,18037.386430,DAVANAGERE,379120.493608,0.720680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41256,1.0,1.0,1.0,4199.0,383.098388,148.020607,3.159514,0.987727,2.646305,5781.521781,5445.227279,2037.447645,KALABURAGI,29036.904194,0.055005
41257,1.0,1.0,1.0,49358.0,30981.381240,30897.307860,3.159514,0.987727,2.646305,50940.521781,36043.510131,32786.734898,YADGIRI,283312.135642,0.538503
41258,1.0,1.0,1.0,115246.0,41912.073070,39866.073200,3.159514,0.987727,2.646305,116828.521781,46974.201961,41755.500238,RAICHUR,526016.829590,1.000000
41259,1.0,1.0,1.0,50753.0,14075.849460,7221.642581,3.159514,0.987727,2.646305,52335.521781,19137.978351,9111.069619,KOPPAL,208368.583671,0.396000


In [54]:
stress_values_list = []
stability_values_list = []

# Define batch size
batch_size = case_file_length

# Iterate over the New_Production column in batches of 31
num_batches = len(result_df) // batch_size

for i in range(num_batches):
    # Get the batch of 31 production values
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch = normalize_list(result_df['New_Production'][start_idx:end_idx].values)
    
    # Calculate the stress and stability values for the batch
    stress_batch, stability_batch = calculate_stress_stability(batch, adj_matrix_np)
    
    stress_values_list.extend(stress_batch)
    stability_values_list.extend(stability_batch)


print(num_batches)
print(len(stress_batch))
print(len(stress_values_list), len(stability_values_list))

result_df['Stress_Values'] = stress_values_list
result_df['Stability_Values'] = stability_values_list

161051
28
4509428 4509428


In [55]:
result_df.columns

Index(['Sensitivity_SowingSeedsDistributed_Maize_320',
       'Sensitivity_LivestockCensus_2012_Cattle_354',
       'Sensitivity_AreaUnderCereals_Maize_193', 'Sensitivity_TotalNPK_315',
       'Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182',
       'SowingSeedsDistributed_Maize_320', 'LivestockCensus_2012_Cattle_354',
       'AreaUnderCereals_Maize_193', 'TotalNPK_315',
       'NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182',
       'Beta_0', 'Beta_1', 'Beta_2', 'Beta_3', 'Beta_4', 'Beta_5', 'New_F1',
       'New_F2', 'New_F3', 'New_F4', 'New_F5', 'District', 'New_Production',
       'Stress_Values', 'Stability_Values'],
      dtype='object')

In [56]:
viz_df = result_df[['District','Sensitivity_SowingSeedsDistributed_Maize_320','Sensitivity_LivestockCensus_2012_Cattle_354','Sensitivity_AreaUnderCereals_Maize_193',
                    'Sensitivity_TotalNPK_315',	'Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182','New_Production','Stability_Values']].copy()

abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
    'Vijayanagara' : 'VJN'
}

viz_df['Abbreviation'] = viz_df['District'].map(abbreviation_mapping)

legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                    for abb, full_form in zip(viz_df['Abbreviation'].unique(), viz_df['District'].unique())]

In [59]:
viz_df['New_Production'] = viz_df['New_Production'].apply(lambda x: max(0, x))

viz_df['Percentage Change'] = viz_df['New_Production'] - np.tile(y.values, len(viz_df) // case_file_length + 1)[:len(viz_df)]
for i in range(0, len(viz_df), case_file_length):
    viz_df.loc[i:i+case_file_length-1, 'Percentage Change'] = viz_df.loc[i:i+case_file_length-1, 'Percentage Change'] * 100 / y.values + 1e-9
    viz_df.loc[i:i+case_file_length-1, 'Normalized Percentage Change'] = normalize_list(viz_df.loc[i:i+case_file_length-1, 'Percentage Change'])


viz_df['Percentage Change'] = viz_df['Percentage Change'].replace([np.inf, -np.inf], 0)

# viz_df['Normalized Percentage Change'] = normalize_list(viz_df['Percentage Change'])


In [61]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
import plotly.graph_objects as go

def update_plot(alpha1, alpha2, alpha3, alpha4, alpha5):
    
    tolerance = 1e-5 
    filtered_df = viz_df[
        (viz_df['Sensitivity_SowingSeedsDistributed_Maize_320'].between(alpha1 - tolerance, alpha1 + tolerance)) &
        (viz_df['Sensitivity_LivestockCensus_2012_Cattle_354'].between(alpha2 - tolerance, alpha2 + tolerance)) &
        (viz_df['Sensitivity_AreaUnderCereals_Maize_193'].between(alpha3 - tolerance, alpha3 + tolerance)) & 
        (viz_df['Sensitivity_TotalNPK_315'].between(alpha4 - tolerance, alpha4 + tolerance)) &
        (viz_df['Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea_182'].between(alpha5 - tolerance, alpha5 + tolerance))
    ]

    # Create a scatter plot with filtered data
    fig = px.scatter(filtered_df, x='Stability_Values', y='Percentage Change', title='Maize Percentage Change vs Stability',
                     labels={'Percentage Change': 'Percentage Change', 'Stability_Values': 'Stability'},
                     text='Abbreviation', opacity=1, color_discrete_sequence=['darkblue'])

    fig.update_traces(textposition='top center', textfont=dict(size=11))

    # Update layout to add annotations and legends
    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.25,
                showarrow=False,
                text=f'α<sub>MS</sub>: {alpha1:.2f}, α<sub>LV</sub>: {alpha2:.2f}, α<sub>AM</sub>: {alpha3:.2f}, α<sub>NPK</sub>: {alpha4:.2f}, α<sub>TBW</sub>: {alpha5:.2f}',
                xref="paper",
                yref="paper",
                font=dict(size=13)
            )
        ],
        legend_title_text='Districts',
        legend_title=dict(font=dict(size=12)),
    )

    # Adding custom legends with abbreviations and full forms
    legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                     for abb, full_form in zip(filtered_df['Abbreviation'].unique(), filtered_df['District'].unique())]

    for label in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(color='darkblue'),
            name=f"{label['Abbreviation']}: {label['Full Form']}"
        ))

    # Adding average lines for stability and percentage change
    avg_stability = filtered_df['Stability_Values'].mean()
    avg_precent = filtered_df['Percentage Change'].mean()

    fig.add_shape(
        type='line',
        x0=avg_stability, y0=0, x1=avg_stability, y1=1,
        line=dict(color='red', width=1.5, dash='dash'),
        yref='paper'
    )

    fig.add_shape(
        type='line',
        x0=0, y0=avg_precent, x1=1, y1=avg_precent,
        line=dict(color='red', width=1.5, dash='dash'),
        xref='paper'
    )

    # Add annotations for average values
    fig.add_trace(go.Scatter(
        x=[avg_stability],
        y=[-0.07],
        text=[f'Avg Stability: {avg_stability:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    fig.add_trace(go.Scatter(
        x=[0.3],
        y=[avg_precent-0.04],
        text=[f'Avg Impact: {avg_precent:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    clear_output()

    display(fig)

# Creating sliders for interaction
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αMS:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αLV:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αAM:')
alpha4_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
alpha5_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αTBW:')

# Creating interactive plot
interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider, alpha4=alpha4_slider, alpha5=alpha5_slider)

# Create update button to trigger plot update
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value, alpha5_slider.value))

# Display the button and sliders
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider, alpha4_slider, alpha5_slider])


interactive(children=(FloatSlider(value=0.5, description='αMS:', max=1.0), FloatSlider(value=0.5, description=…

In [70]:
result_df.to_csv('Rice_Prescriptive_5000.csv',index=False)